In [2]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
import os
from dotenv import load_dotenv
from sodapy import Socrata
import chromedriver_binary
#from src.cleaning3 import *
import folium
from folium.plugins import HeatMapWithTime as HMWT
from folium.plugins import HeatMap as HM
from folium import plugins
import warnings
warnings.simplefilter('ignore')

In [3]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time 
import urllib3


In [4]:
df=pd.read_csv('../DATA/DataCo/token2.csv')
df.head()

,product,category,date,department,ip,year,month_
0,adidas Brazuca 2017 Official Match Ball,baseball & softball,01/09/2017,fitness,37.97.182.65,2017,9
1,The North Face Women's Recon Backpack,hunting & shooting,01/09/2017,fan shop,206.56.112.1,2017,9
2,adidas Kids' RG III Mid Football Cleat,featured shops,01/09/2017,apparel,215.143.180.0,2017,9
3,Under Armour Men's Compression EV SL Slide,electronics,01/09/2017,footwear,206.56.112.1,2017,9
4,Pelican Sunstream 100 Kayak,water sports,01/09/2017,fan shop,136.108.56.242,2017,9


In [5]:
df.shape

(469977, 7)

In [6]:
pd.set_option('display.max_rows', None)

ipp=list(df['ip'].value_counts()[df['ip'].value_counts()> 100].keys()) #> de 100 consultas desde la IP

In [7]:
sum(list(df['ip'].value_counts()[df['ip'].value_counts()> 100]))

407615

In [8]:
len(df['ip'].value_counts()[df['ip'].value_counts()> 100].keys())

1758

In [9]:
pd.set_option('display.max_rows', None)
len(ipp)

1758

def geo(ip):
    url=f'https://tools.keycdn.com/geo?host={ip}'
    html=req.get(url).content
    soup=bs(html, 'html.parser')
    tabla=soup.find('div', {'id': 'geoResult'}) 
    conexion=[e.text for e in tabla.find('dd', {'class': 'col-8 text-monospace'})]
    return conexion                                                           #IP SCRAPING

In [10]:
'''def geo(ip):
    url=f'https://tools.keycdn.com/geo?host={ip}'
    html=req.get(url).content
    soup=bs(html, 'html.parser')
    tabla=soup.find('div', {'id': 'geoResult'}) 
    conexion=[e.text for e in tabla.find('dl', {'class': 'row mb-0'})][:11]
    return conexion      '''                                                     #IP SCRAPING ip3

"def geo(ip):\n    url=f'https://tools.keycdn.com/geo?host={ip}'\n    html=req.get(url).content\n    soup=bs(html, 'html.parser')\n    tabla=soup.find('div', {'id': 'geoResult'}) \n    conexion=[e.text for e in tabla.find('dl', {'class': 'row mb-0'})][:11]\n    return conexion      "

In [11]:
'''ip3=[]
for ip in ipp:
    print(geo(ip))
    ip3.append(geo(ip))'''

'ip3=[]\nfor ip in ipp:\n    print(geo(ip))\n    ip3.append(geo(ip))'

In [12]:
ip3[:2] #las ips con la información de region, country, etc...

NameError: name 'ip3' is not defined

In [ ]:
cols11=['1', '2', 'city', '4', 'region', '6', 'postal_code', '8', 'country', '10', 'continent']

In [ ]:
dt=pd.DataFrame(ip3,columns=cols11)
dt.head(5)

In [ ]:
dt=dt.drop(columns=['1','2','4','6','8','10','postal_code','continent'])

In [ ]:
dt['IP']=ipp

In [ ]:
dt.head()

In [ ]:
def change(x):
    try:
        if '(America/Chicago)' in x:
            return 'United States (US)'
        elif '(Europe/Paris)' in x:
            return 'Europe'
        elif '(Europe/Stockholm)' in x:
            return 'Sweden'
        elif '(America/Sao_Paulo)' in x:
            return 'Brazil'
        elif '(Europe/London)' in x:
            return 'United Kingdom'
        elif '(Asia/Shanghai)' in x:
            return 'China (CN)'
        elif '(Asia/Tokyo)' in x:
            return 'Japan'
        elif '(Europe/Brussels)' in x:
            return 'Belgium'
        elif '(America/Lima)' in x:
            return 'Peru'
        elif '(Europe/Berlin)' in x:
            return 'Germany'
        elif '2022'  in x:
            return 'Unknown'
        elif 'lat' in x:
            return 'Unknown'
        else:
            return x
    except:
        'Unknown'

In [ ]:
dt['country']=dt['country'].apply(change)

In [ ]:
dt.head()

In [ ]:
import re
def regexx(x):
    try:
        return re.findall('[^(.*)]+',x)[0]
    except:
        return x

In [ ]:
regexx('Seoul (11)')

In [ ]:
dt['region']=dt['region'].apply(regexx)
dt['city']=dt['region'].apply(regexx)
dt['country']=dt['country'].apply(regexx)

In [ ]:
dt.head()

In [776]:
dt.to_csv('IP.csv', index=False)

In [13]:
df=pd.read_csv('../DATA/DataCo/IP.csv')
df.head()

,city,region,country,IP
0,Virginia,Virginia,United States,157.21.93.193
1,Europe,Europe,Europe,138.21.216.113
2,Tel Aviv,Tel Aviv,Asia,77.137.114.147
3,Krasnodarskiy Kray,Krasnodarskiy Kray,Russia,83.234.215.133
4,Shanghai,Shanghai,Asia,47.102.94.70


In [17]:
df.city.value_counts()

North America                              561
Asia                                       179
Europe                                     154
Africa                                      28
South America                               21
Virginia                                    21
California                                  20
Guangdong                                   16
New York                                    15
Texas                                       15
Oceania                                     14
Tokyo                                       14
Illinois                                    13
Sao Paulo                                   11
Seoul                                       11
Florida                                     10
Beijing                                      9
Jiangsu                                      9
Gyeonggi-do                                  8
Iowa                                         8
Victoria                                     8
Washington   

In [779]:
dt.country.value_counts()

United States          739
Unknown                217
Asia                   118
China                   68
Japan                   45
Japan                   43
United Kingdom          34
South Korea             33
Germany                 32
United Kingdom          27
France                  26
Russia                  22
Italy                   22
Germany                 21
Brazil                  20
Australia               18
South America           18
Africa                  17
Spain                   16
Europe                  15
Argentina               12
Europe                  11
Canada                  10
Brazil                  10
Poland                  10
Colombia                 9
Mexico                   9
Netherlands              9
India                    9
Sweden                   8
South Africa             7
Turkey                   6
Denmark                  6
Sweden                   5
Finland                  5
Belgium                  4
Portugal                 4
M

In [753]:
#debajo el primer intento de scrapeo

In [ ]:
ip2[:10] #solo son los primeros valores 'country'

In [378]:
'''ip2=[]
for ip in ipp:
    print(geo(ip))
    ip2.append(geo(ip)) #solo con los country'''

['Virginia Beach']
['France (FR)']
['Tel Aviv']
['Tuapse']
['Shanghai']
['Japan (JP)']
['Japan (JP)']
['Scarborough']
['Tunis']
['United States (US)']
['United States (US)']
['United States (US)']
['Helsinki']
['United States (US)']
['United States (US)']
['Germany (DE)']
['Karasawa']
['Lublin']
['United States (US)']
['10.184.28.55']
['Kunming']
['Warsaw']
['Huntsville']
['United States (US)']
['South Korea (KR)']
['United States (US)']
['Canada (CA)']
['Bow']
['Montreal']
['United States (US)']
['United Kingdom (GB)']
['Rio de Janeiro']
['Rome']
['Japan (JP)']
['Linesville']
['Tokyo']
['Rende']
['United States (US)']
['Germany (DE)']
['United States (US)']
['United States (US)']
['United States (US)']
['Trainou']
['Boardman']
['United States (US)']
['Saint Paul']
['Montreal']
['Brisbane']
['United States (US)']
['United States (US)']
['United States (US)']
['United States (US)']
['South Korea (KR)']
['Carrollton']
['United States (US)']
['United States (US)']
['United States (US)']
[

['Brisbane']
['United States (US)']
['United States (US)']
['Colona']
['Japan (JP)']
['Rochetaillee']
['El Jem']
['Boardman']
['Bogotá']
['Germany (DE)']
['United States (US)']
['China (CN)']
['Guangzhou']
['United States (US)']
['United States (US)']
['10.234.80.37']
['United States (US)']
['United Kingdom (GB)']
['Izmir']
['Hasseris']
['Fruges']
['Japan (JP)']
['Canada (CA)']
['South Korea (KR)']
['Denver']
['Germany (DE)']
['United States (US)']
['Los Angeles']
['United States (US)']
['Osaka']
['Foz']
['Chongqing']
['Charlotte']
['Philippines (PH)']
['United Kingdom (GB)']
['South Korea (KR)']
['United States (US)']
['United States (US)']
['United States (US)']
['United States (US)']
['Nanjing']
['United States (US)']
['Kaevlinge']
['Erfurt']
['United States (US)']
['Queens']
['Slagelse']
['São Caetano do Sul']
['Germany (DE)']
['Zhangzhou']
['Rotterdam']
['Menai Bridge']
['Tsarskoye Selo']
['Mont-de-Marsan']
['Germany (DE)']
['United States (US)']
['United States (US)']
['United St

['United States (US)']
['United States (US)']
['Syria (SY)']
['Brazil (BR)']
['United States (US)']
['United States (US)']
['Soka']
['Turkey (TR)']
['West Palm Beach']
['United States (US)']
['Shibuya']
['Zimbabwe (ZW)']
['United States (US)']
['Cairo']
['Denmark (DK)']
['United States (US)']
['Gapyeong County']
['Yelin']
['Chengdu']
['Seattle']
['China (CN)']
['Okaya Shi']
['Bunkyo-ku']
['United States (US)']
['Kingston']
['Mogi das Cruzes']
['Paterson']
['United States (US)']
['Bergen']
['Palma']
['United States (US)']
['Hellerup']
['Sutton Coldfield']
['Fergus Falls']
['United States (US)']
['Japan (JP)']
['United States (US)']
['10.118.162.230']
['Tappahannock']
['Greensburg']
['United Kingdom (GB)']
['New Taipei']
['United States (US)']
['Switzerland (CH)']
['Austin']
['Warsaw']
['Tacoma']
['Japan (JP)']
['Areannamkwaengi']
['United States (US)']
['Lowell']
['United States (US)']
['Beilin']
['United States (US)']
['South Korea (KR)']
['Czechia (CZ)']
['United States (US)']
['Goron

['Colombia (CO)']
['Lima']
['Uganda (UG)']
['Guymon']
['United States (US)']
['Tamuning']
['Urk']
['Norway (NO)']
['China (CN)']
['United States (US)']
['Encinitas']
['Palmerston North']
['Puebla City']
['Minsk']
['Santiago']
['China (CN)']
['China (CN)']
['Jerusalem']
['Columbus']
['Escondido']
['United States (US)']
['Kvistgard']
['Bologoye']
['United States (US)']
['Haskovo']
['United Kingdom (GB)']
['Houston']
['South Korea (KR)']
['Shenzhen']
['Wuhan']
['China (CN)']
['United States (US)']
['Central']
["Xi'an"]
['Chicago']
['Taiwan (TW)']
['Tel Aviv']
['United States (US)']
['United States (US)']
['Pittsford']
['United States (US)']
['Hanoi']
['United States (US)']
['United States (US)']
['Erith']
['United States (US)']
['Fairfax']
['Mexico (MX)']
['France (FR)']
['United States (US)']
['Buenos Aires']
['Terbanggi Besar']
['United States (US)']
['United States (US)']
['France (FR)']
['United States (US)']
['United States (US)']
['Chantilly']
['Japan (JP)']
['Guarulhos']
['United S

In [381]:
'''IP_def=[j for i in ip2 for j in i]''' # IP's countries

In [384]:
'''IP_def'''

['Virginia Beach',
 'France (FR)',
 'Tel Aviv',
 'Tuapse',
 'Shanghai',
 'Japan (JP)',
 'Japan (JP)',
 'Scarborough',
 'Tunis',
 'United States (US)',
 'United States (US)',
 'United States (US)',
 'Helsinki',
 'United States (US)',
 'United States (US)',
 'Germany (DE)',
 'Karasawa',
 'Lublin',
 'United States (US)',
 '10.184.28.55',
 'Kunming',
 'Warsaw',
 'Huntsville',
 'United States (US)',
 'South Korea (KR)',
 'United States (US)',
 'Canada (CA)',
 'Bow',
 'Montreal',
 'United States (US)',
 'United Kingdom (GB)',
 'Rio de Janeiro',
 'Rome',
 'Japan (JP)',
 'Linesville',
 'Tokyo',
 'Rende',
 'United States (US)',
 'Germany (DE)',
 'United States (US)',
 'United States (US)',
 'United States (US)',
 'Trainou',
 'Boardman',
 'United States (US)',
 'Saint Paul',
 'Montreal',
 'Brisbane',
 'United States (US)',
 'United States (US)',
 'United States (US)',
 'United States (US)',
 'South Korea (KR)',
 'Carrollton',
 'United States (US)',
 'United States (US)',
 'United States (US)',
 

In [385]:
'''total = zip(ipp,IP_def)
totals=dict(total)
totals'''

{'157.21.93.193': 'Virginia Beach',
 '138.21.216.113': 'France (FR)',
 '77.137.114.147': 'Tel Aviv',
 '83.234.215.133': 'Tuapse',
 '47.102.94.70': 'Shanghai',
 '150.89.112.119': 'Japan (JP)',
 '211.122.14.29': 'Japan (JP)',
 '99.237.181.177': 'Scarborough',
 '102.172.170.187': 'Tunis',
 '69.214.176.127': 'United States (US)',
 '199.52.45.6': 'United States (US)',
 '67.207.55.106': 'United States (US)',
 '62.248.229.117': 'Helsinki',
 '140.34.153.203': 'United States (US)',
 '19.26.1.207': 'United States (US)',
 '134.2.143.5': 'Germany (DE)',
 '210.138.158.124': 'Karasawa',
 '89.66.129.112': 'Lublin',
 '154.83.178.158': 'United States (US)',
 '10.184.28.55': '10.184.28.55',
 '183.225.48.201': 'Kunming',
 '37.30.32.216': 'Warsaw',
 '158.135.5.17': 'Huntsville',
 '50.158.77.52': 'United States (US)',
 '219.252.175.214': 'South Korea (KR)',
 '165.138.191.157': 'United States (US)',
 '142.15.244.133': 'Canada (CA)',
 '73.47.232.83': 'Bow',
 '64.228.48.11': 'Montreal',
 '34.5.176.235': 'Unit

In [386]:
def match(x):
    try:
        return totals[f"{x}"]
    except:
        return "Unknown"

In [387]:
'''df1['country_ip']=df1['ip'].apply(match)
df1.head(5)'''

,product,category,date,department,ip,year,month_,country_ip
0,adidas Brazuca 2017 Official Match Ball,baseball & softball,01/09/2017,fitness,37.97.182.65,2017,9,Netherlands (NL)
1,The North Face Women's Recon Backpack,hunting & shooting,01/09/2017,fan shop,206.56.112.1,2017,9,United States (US)
2,adidas Kids' RG III Mid Football Cleat,featured shops,01/09/2017,apparel,215.143.180.0,2017,9,United States (US)
3,Under Armour Men's Compression EV SL Slide,electronics,01/09/2017,footwear,206.56.112.1,2017,9,United States (US)
4,Pelican Sunstream 100 Kayak,water sports,01/09/2017,fan shop,136.108.56.242,2017,9,United States (US)


In [388]:
'''df2['country_ip']=df2['ip'].apply(match)
df2.head(5)'''

,product,category,date,department,ip,year,month_,country_ip
100000,SOLE E25 Elliptical,basketball,25/10/2017,fitness,56.16.86.156,2017,10,United States (US)
100001,Field & Stream Sportsman 16 Gun Fire Safe,fishing,25/10/2017,fan shop,213.194.136.120,2017,10,Marchena
100002,Perfect Fitness Perfect Rip Deck,cleats,25/10/2017,apparel,60.14.203.94,2017,10,Beijing
100003,Stiga Master Series ST3100 Competition Indoor,hockey,25/10/2017,fitness,124.176.103.135,2017,10,Melbourne
100004,adidas Kids' RG III Mid Football Cleat,featured shops,25/10/2017,apparel,62.168.81.99,2017,10,Hrinova


In [389]:
'''df3['country_ip']=df3['ip'].apply(match)
df3.head(5)'''

,product,category,date,department,ip,year,month_,country_ip
200000,The North Face Women's Recon Backpack,hunting & shooting,23/12/2017,fan shop,20.19.190.144,2017,12,United States (US)
200001,Nike Men's Free TR 5.0 TB Training Shoe,as seen on tv!,23/12/2017,footwear,112.70.47.50,2017,12,Ōtsu
200002,Cleveland Golf Women's 588 RTX CB Satin Chrom,golf apparel,23/12/2017,outdoors,135.44.141.207,2017,12,United States (US)
200003,adidas Brazuca 2017 Official Match Ball,baseball & softball,23/12/2017,fitness,105.130.122.91,2017,12,Temara
200004,Under Armour Girls' Toddler Spine Surge Runni,shop by sport,23/12/2017,golf,14.18.78.195,2017,12,Guangzhou


In [390]:
'''df4['country_ip']=df4['ip'].apply(match)
df4.head(5)'''

,product,category,date,department,ip,year,month_,country_ip
300000,Nike Men's Fingertrap Max Training Shoe,soccer,05/01/2018,fitness,149.4.46.165,2018,1,Queens
300001,TaylorMade 2017 Purelite Stand Bag,golf gloves,05/01/2018,outdoors,14.26.36.114,2018,1,Shenzhen
300002,Glove It Women's Mod Oval Golf Glove,golf balls,05/01/2018,outdoors,104.219.148.203,2018,1,Lost Nation
300003,Nike Men's Free 5.0+ Running Shoe,cardio equipment,05/01/2018,footwear,71.36.8.24,2018,1,Columbia City
300004,Under Armour Hustle Storm Medium Duffle Bag,fitness accessories,05/01/2018,footwear,63.10.161.243,2018,1,United States (US)


In [391]:
'''total=pd.concat([df1,df2,df3,df4],axis=0).reset_index()'''

In [ ]:
# seleccionar unicamente 1